# 8.2 Reward Model

# 8.3 SFT: Supervised Fine-Tuning

## quick start

In [ ]:
from datasets import load_dataset
from trl import SFTTrainer

dataset = load_dataset("imdb", split="train")

trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
)

trainer.train()

/home/nsw/.pyenv/versions/3.10.13/envs/main/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


## setup chat format

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import setup_chat_format

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
print("before:", tokenizer.chat_template)

# Set up the chat format with default 'chatml' format
model, tokenizer = setup_chat_format(model, tokenizer)
print("after:", tokenizer.chat_template)

before: None
after: {% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}


## formatting func

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
        output_texts.append(text)
    return output_texts

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
)

trainer.train()

## packing

In [ ]:
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    dataset_text_field="text",
    packing=True
)

trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]

## model_init_kwargs

In [ ]:
import torch

trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    dataset_text_field="text",
    model_init_kwargs={
        "torch_dtype": torch.bfloat16,
        "attn_implementation": "flash_attention_2"
    },
)

trainer.train()

You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


## peft_config

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    dataset_text_field="text",
    peft_config=peft_config
)

# trainer.train()

/home/nsw/.pyenv/versions/3.10.13/envs/main/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    dataset_text_field="text",
    model_init_kwargs={
        "torch_dtype": torch.bfloat16,
        "load_in_4bit": True,
        "attn_implementation": "flash_attention_2"
    },
    peft_config=peft_config,
)

# trainer.train()

/home/nsw/.pyenv/versions/3.10.13/envs/main/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


## model_config

In [ ]:
from trl import ModelConfig, SFTTrainer, get_kbit_device_map, get_peft_config, get_quantization_config

model_config = ModelConfig(
    model_name_or_path="facebook/opt-350m",
    attn_implementation="flash_attention_2",
    load_in_4bit=True,
    use_peft=True,
)
model_config

ModelConfig(model_name_or_path='facebook/opt-350m', model_revision='main', torch_dtype=None, trust_remote_code=False, attn_implementation='flash_attention_2', use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, lora_target_modules=None, lora_modules_to_save=None, lora_task_type='CAUSAL_LM', load_in_8bit=False, load_in_4bit=True, bnb_4bit_quant_type='nf4', use_bnb_nested_quant=False)

In [ ]:
quantization_config = get_quantization_config(model_config)
quantization_config

BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "float32",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": false,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}

In [ ]:
get_kbit_device_map()

{'': 0}

In [ ]:
peft_config = get_peft_config(model_config)
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='CAUSAL_LM', inference_mode=False, r=16, target_modules=None, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False)

In [ ]:
torch_dtype = (
    model_config.torch_dtype
    if model_config.torch_dtype in ["auto", None]
    else getattr(torch, model_config.torch_dtype)
)
model_kwargs = dict(
    revision=model_config.model_revision,
    trust_remote_code=model_config.trust_remote_code,
    attn_implementation=model_config.attn_implementation,
    torch_dtype=torch_dtype,
    device_map=get_kbit_device_map() if quantization_config is not None else None,
    quantization_config=quantization_config,
)

trainer = SFTTrainer(
    model=model_config.model_name_or_path,
    train_dataset=dataset,
    dataset_text_field="text",
    model_init_kwargs=model_kwargs,
    peft_config=peft_config,
)

## neftune_noise_alpha

In [ ]:
trainer = SFTTrainer(
    "facebook/opt-350m",
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=512,
    neftune_noise_alpha=5,
)
# trainer.train()

/home/nsw/.pyenv/versions/3.10.13/envs/main/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:166: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

# 8.4 PPO: Proximal Policy Optimization


In [ ]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/cherry_picked_prompts", split="train")
dataset = dataset.rename_column("prompt", "query")
dataset = dataset.remove_columns(["meta", "completion"])
dataset[0]

{'query': 'Explain the moon landing to a 6 year old in a few sentences.'}

In [ ]:
from transformers import AutoTokenizer
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer

config = PPOConfig(
    model_name="gpt2",
    learning_rate=1.41e-5,
)
model = (
    AutoModelForCausalLMWithValueHead
    .from_pretrained(config.model_name)
)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

# TDDO: 이거 eos 붙여서 학습해야하나?
# tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline

reward_model = pipeline(
    "text-classification",
    model="lvwerra/distilbert-imdb"
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def tokenize(sample):
    # eos 붙여서?
    sample["input_ids"] = tokenizer.encode(sample["query"])
    return sample

dataset = dataset.map(tokenize, batched=False)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [ ]:
from trl import PPOTrainer

ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    dataset=dataset,
    tokenizer=tokenizer,
)

In [ ]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

In [ ]:
from tqdm import tqdm


epochs = 10
for epoch in tqdm(range(epochs), "epoch: "):
    for batch in tqdm(ppo_trainer.dataloader):
        query_tensors = batch["input_ids"]

        # Rollout: 학습할 모델로 문장 생성
        response_tensors = ppo_trainer.generate(
            query_tensors,
            **generation_kwargs
        )
        batch["response"] = [
            tokenizer.decode(r.squeeze())
            for r in response_tensors
        ]

        # Evaluate: Reward 모델로 점수 부여
        texts = [q + r for q, r in zip(batch["query"], batch["response"])]
        pipe_outputs = reward_model(texts)
        rewards = [
            torch.tensor(output[1]["score"])
            for output in pipe_outputs
        ]

        # Optimization: ppo 학습 진행
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)

# 모델 저장
ppo_trainer.save_pretrained("my_ppo_model")

In [ ]:
ppo_trainer.tr

# 8.5 Best of N Sampling

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, GenerationConfig
from trl import AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler
from trl.extras import BestOfNSampler

ref_model_name = "gpt2"
reward_model_name = "gpt2"
device = torch.device("cuda")
ref_model = (
    AutoModelForCausalLMWithValueHead
    .from_pretrained(ref_model_name)
    .to(device)
)
tokenizer = AutoTokenizer.from_pretrained(ref_model_name)

reward_pipe = pipeline(
    "sentiment-analysis",
    model=reward_model_name,
    device=device
)

def queries_to_scores(list_of_strings):
    return [output["score"] for output in reward_pipe(list_of_strings)]

best_of_n = BestOfNSampler(
    ref_model,
    tokenizer,
    queries_to_scores,
    length_sampler=LengthSampler(10, 128),
    sample_size=5,
    n_candidates=2,
    generation_config=GenerationConfig(
        min_length= -1,
        top_k=0.0,
        top_p= 1.0,
        do_sample= True,
        pad_token_id=tokenizer.eos_token_id
    ),
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
result = best_of_n.generate(
    tokenizer("what is love?", return_tensors="pt").input_ids[0],
    device=device
)

for r in result[0]:
    print(r)
    print("=" * 50, "\n")

what is love? Love? Where can I find it?

Love love, what are you love loving?

Love love, what is the Holiness to you?

Love feeling love, what is the Glory of God in you?

Love love, what does it mean to love?

Love. Love who has

what is love? How has love been changed? How things would change under God? Do God's desires continue with the child? Does love diminish with society's powers? Should people control themselves? At times these questions have seemed theological, yet of all the demanding questions that Jesus confronts, surely we get the most devastated reading.

John Piper's



# 8.6 DPO: Directi Preference Optimization


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "trl-internal-testing/hh-rlhf-trl-style",
    split="train",
)
dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 160800
})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
ref_model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
ref_model = AutoModelForCausalLM.from_pretrained(ref_model_name)

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.chat_template is None:
    tokenizer.chat_template = "{% for message in messages %}{{message['role'] + ': ' + message['content'] + '\n\n'}}{% endfor %}{{ eos_token }}"

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)
print(dataset[1]['chosen'])

Map (num_proc=2):   0%|          | 0/160800 [00:00<?, ? examples/s]

user: What kind of noises did dinosaurs make?

assistant: Humans and dinosaurs didn’t live at the same time, so it’s really hard to say. The best place to find out what noises dinosaurs made would be

user: yes they did

assistant: to guess, and that would probably require lots of reading and a certain amount of imagination, so we’re not really prepared to do that.

user: you cant read

assistant: You can read?

<|endoftext|>


/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
from transformers import TrainingArguments
from trl import DPOTrainer

args = TrainingArguments(
    logging_dir="logs",
    output_dir="ckpt",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,

)

trainer = DPOTrainer(
        model,
        ref_model,
        tokenizer=tokenizer,
        args=args,
        train_dataset=dataset,
        max_length=512,
        max_prompt_length=512,
        dataset_num_proc=2,
        remove_unused_columns=False,
    )

trainer.train()

# 8.7 KTO: Kahneman-Tversky Optimization

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "trl-lib/kto-mix-14k",
    split="train",
)
dataset

Dataset({
    features: ['prompt', 'completion', 'label'],
    num_rows: 13500
})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "trl-lib/qwen1.5-1.8b-sft"
ref_model_name = "trl-lib/qwen1.5-1.8b-sft"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
ref_model = AutoModelForCausalLM.from_pretrained(ref_model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def process(row):
    row["prompt"] = tokenizer.apply_chat_template(row["prompt"], tokenize=False)
    row["completion"] = tokenizer.apply_chat_template(row["completion"], tokenize=False)
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)
print(dataset[1]['completion'])

Map (num_proc=2):   0%|          | 0/13500 [00:00<?, ? examples/s]

<|im_start|>assistant
No, the answer provided does not directly answer the question regarding the age of Julio Cesar Chavez when he fought Oscar De La Hoya. The provided information describes some general records held by Julio Cesar Chavez throughout his career. To answer your original question, let me provide the relevant information:

Julio Cesar Chavez fought Oscar De La Hoya on June 7, 1996, in a match called "Ultimate Glory." Chavez was born on July 12, 1962. To calculate his age at the time of the fight, we need to find the difference between the fight date and his birthdate.

From July 12, 1962, to June 7, 1996, there are:
- 33 years (from 1962 to 1995)
- An additional year from his birthday in 1995 (July 12, 1995) to the fight date in 1996 (June 7, 1996), which is approximately 10 months and 26 days.

Therefore, Julio Cesar Chavez was about 33 years and 10 months old when he fought Oscar De La Hoya.<|im_end|>



/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
from trl import KTOTrainer, KTOConfig

args = KTOConfig(
    logging_dir="logs",
    output_dir="ckpt",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,

    max_length=512,
    max_prompt_length=512,
    remove_unused_columns=False,
    dataset_num_proc=2,

    beta=0.1,
    desirable_weight=1.0,
    undesirable_weight=1.0,
)

trainer = KTOTrainer(
    model,
    ref_model,
    args=args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

trainer.train()

Tokenizing train dataset:   0%|          | 0/13500 [00:00<?, ? examples/s]

Extracting KL train dataset:   0%|          | 0/13500 [00:00<?, ? examples/s]

Processing tokenized train dataset (num_proc=2):   0%|          | 0/13500 [00:00<?, ? examples/s]

/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Processing tokenized train KL dataset (num_proc=2):   0%|          | 0/13500 [00:00<?, ? examples/s]

/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


Filtering desirable examples (num_proc=2):   0%|          | 0/13500 [00:00<?, ? examples/s]

Filtering undesirable examples (num_proc=2):   0%|          | 0/13500 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.14 GiB (GPU 0; 11.99 GiB total capacity; 40.06 GiB already allocated; 0 bytes free; 40.59 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# 8.8 CPO: Contrastive Preference Optimization

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "trl-internal-testing/hh-rlhf-trl-style",
    split="train",
)
dataset

Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 160800
})

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if tokenizer.chat_template is None:
    tokenizer.chat_template = "{% for message in messages %}{{message['role'] + ': ' + message['content'] + '\n\n'}}{% endfor %}{{ eos_token }}"

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset = dataset.map(
    process,
    num_proc=2,
    load_from_cache_file=False,
)

Map (num_proc=2):   0%|          | 0/160800 [00:00<?, ? examples/s]

/home/nsw/.pyenv/versions/3.10.13/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
from trl import CPOConfig, CPOTrainer

args = CPOConfig(
    logging_dir="logs",
    output_dir="ckpt",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    optim="adamw_torch",
    logging_steps=100,

    max_length=512,
    max_prompt_length=512,
    dataset_num_proc=2,
    remove_unused_columns=False,

    beta=0.1,
)

trainer = CPOTrainer(
    model,
    tokenizer=tokenizer,
    args=args,
    train_dataset=dataset,
)

trainer.train()